
1)Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru 
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям. 
2)Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)
3)Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import requests
import re
from bs4 import BeautifulSoup as bs

In [2]:
# functions: choosing the job and getting the link to search vacancies by job's name
def get_name():
    job = ('Программист')
    return job

def get_link():
    link='https://www.superjob.ru/vacancy/search/?keywords='+get_name()
    return link

# getting parsed html code
def parsing_page(link = get_link()):
    response = requests.get(link)
    site_text = response.text
    parsed_page = bs(site_text, 'html.parser')
    return parsed_page
    
# getting an iterable object of 30 vacancies from the site's page
def searching_vacancy_block(parsed_site):
    vacancy_blocks = parsed_site.findAll('div', attrs={'class': '_3zucV _2GPIV i6-sc _3VcZr'})
    return vacancy_blocks

# getting a link to the next page of vacancies from the parsed html code
def searching_link(parsed_page):
    searching_link = parsed_page.find('a', rel='next')
    link_2 = searching_link['href']
    new_link = f'https://www.superjob.ru{link_2}'
    return new_link

# extracting data on each vacancy from an object of 30 vacancies and restructuring it to pandas.dataframe
def extracting_vacancies(vacancy_block):
    vacancy_name = []
    vacancy_link = []
    vacancy_salary = []
    for vacancy in vacancy_block:
        # extracting vacancy's link
        link = vacancy.find('a', attrs={'href': re.compile(r'^/vakansii/')})
        l = link['href']
        # storing it into a list variable
        vacancy_link.append(l)
        # extracting vacancy's name
        job_pos = vacancy.find('div', attrs={'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
        j = job_pos.get_text()
        # storing it into a list variable
        vacancy_name.append(j)
        # extracting the salary offered
        job_sal = vacancy.find('span', attrs={'class': re.compile("salary")})
        s = job_sal.get_text()
        s = s.replace('\xa0', ' ')
        # storing it into a list variable
        vacancy_salary.append(s)
        # collecting lists
        #vacancy_list = [vacancy_name, vacancy_link, vacancy_salary]
        # lists of data into pandas.dataframe
        vacancy_df = pd.DataFrame({'Название вакансии': vacancy_name, 'Ссылка на вакансию': vacancy_link, 'Заработная плата': vacancy_salary},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата'])
    return vacancy_df



In [3]:
# forming an empty dataframe
vacancies_df = pd.DataFrame({'Название вакансии': [], 'Ссылка на вакансию': [], 'Заработная плата': []},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата'])

# calling function to get a parsed first page with vacancies
parsed_page = parsing_page()

# calling a function to get an object with 30 vacancies from the parsed page
vacancy_block = searching_vacancy_block(parsed_page)


In [4]:
# a cicle of collecting data from the first page, getting a link to the next page and collecting data from it too
while parsed_page.find('a', rel='next'):
    vacancy_df = extracting_vacancies(vacancy_block)
    # appending formed dataframes from each page until there is no "next" page to parse
    vacancies_df = vacancies_df.append(vacancy_df)

    parsed_page = parsing_page(searching_link(parsed_page))
    vacancy_block = searching_vacancy_block(parsed_page)

        

In [5]:
vacancies_df.head(100)

,Название вакансии,Ссылка на вакансию,Заработная плата
0,Программист 1С,/vakansii/programmist-1s-32109978.html,150 000 ₽
1,Инженер-программист,/vakansii/inzhener-programmist-32374258.html,от 52 000 ₽
2,Программист ABAP,/vakansii/programmist-abap-26387516.html,По договорённости
3,Программист 1С,/vakansii/programmist-1s-31878984.html,По договорённости
4,Ведущий программист 1С (г. Колпино),/vakansii/veduschij-programmist-1s-30544443.html,По договорённости
5,Программист PHP,/vakansii/programmist-php-32364807.html,По договорённости
6,Программист-наладчик токарных станков ЧПУ,/vakansii/programmist-naladchik-tokarnyh-stank...,По договорённости
7,"Ведущий специалист в области IT, программист 1С",/vakansii/veduschij-specialist-v-oblasti-it-32...,от 60 000 ₽
8,Программист PHP,/vakansii/programmist-php-32334457.html,от 70 000 ₽
9,Инженер-программист,/vakansii/inzhener-programmist-32115513.html,По договорённости
